In [1]:
from blackjack import BlackJack
import numpy as np

In [2]:
game = BlackJack()

Object created


In [7]:
[a for (a,_,_) in game.states]

[13,
 13,
 18,
 8,
 12,
 6,
 20,
 8,
 4,
 19,
 15,
 17,
 17,
 10,
 12,
 18,
 14,
 16,
 11,
 15,
 13,
 6,
 18,
 4,
 8,
 12,
 6,
 19,
 20,
 17,
 19,
 15,
 5,
 17,
 17,
 7,
 10,
 12,
 18,
 14,
 16,
 11,
 15,
 13,
 13,
 6,
 18,
 20,
 8,
 4,
 19,
 20,
 5,
 17,
 17,
 7,
 5,
 9,
 12,
 18,
 18,
 14,
 16,
 11,
 15,
 13,
 13,
 18,
 19,
 6,
 21,
 20,
 8,
 14,
 4,
 16,
 19,
 5,
 17,
 7,
 9,
 18,
 16,
 20,
 14,
 16,
 11,
 15,
 11,
 13,
 21,
 13,
 18,
 19,
 16,
 20,
 8,
 14,
 4,
 5,
 17,
 7,
 9,
 18,
 14,
 16,
 20,
 14,
 11,
 15,
 13,
 13,
 19,
 21,
 4,
 8,
 14,
 16,
 20,
 4,
 12,
 5,
 9,
 17,
 7,
 18,
 14,
 15,
 16,
 17,
 20,
 10,
 11,
 12,
 15,
 19,
 13,
 21,
 4,
 14,
 16,
 20,
 12,
 17,
 7,
 12,
 5,
 9,
 15,
 17,
 18,
 20,
 14,
 10,
 15,
 16,
 12,
 20,
 10,
 11,
 19,
 13,
 21,
 20,
 14,
 16,
 12,
 6,
 5,
 17,
 7,
 9,
 18,
 19,
 15,
 17,
 20,
 10,
 12,
 13,
 21,
 19,
 13,
 21,
 16,
 14,
 16,
 18,
 12,
 5,
 6,
 7,
 9,
 18,
 19,
 15,
 16,
 17,
 20,
 10,
 12,
 19,
 13,
 21,
 19,
 14,
 16,
 13,
 14,
 